In [ ]:
import socket

NB='test'
DESCRIPTION='test notebook'
HOST = socket.gethostname()
HOST

In [2]:
import warnings
import os
from pathlib import Path

import mlflow
from dataclasses import dataclass
from dataclasses import asdict

import pandas as pd
#import lightgbm as lgb
#import xgboost as xgb
from catboost import CatBoostClassifier, CatBoostRegressor, Pool
import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import accuracy_score, mean_squared_error, roc_auc_score, average_precision_score, mean_squared_log_error
from sklearn.preprocessing import LabelEncoder

#from my_utils import get_notebook_path

warnings.simplefilter('ignore')
pd.set_option('display.max_columns', 25)
pd.set_option('display.max_rows', 25)

In [3]:
ROOT_DIR = Path('./')
DATA_DIR = ROOT_DIR / Path('data')

In [4]:
class Config:
    SR = 48000
    N_LABEL = 24
    N_FOLD = 5
    RANDOM_SATE = 42
    LR = 0.15
    PATIENCE = 30
    EPOCH = 300
    BATCH_SIZE = 6
    N_REPEATS = 3
    SKIP_EVALUATE_NUM = 5

In [5]:
train_df = pd.read_csv(DATA_DIR / Path('train.csv'))
test_df = pd.read_csv(DATA_DIR / Path('test.csv'))

display(train_df.shape)
display(train_df.head(5))
display(test_df.shape)
display(test_df.head(5))

(891, 12)

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


(418, 11)

,PassengerId,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,892,3,"Kelly, Mr. James",male,34.5,0,0,330911,7.8292,NaN,Q
1,893,3,"Wilkes, Mrs. James (Ellen Needs)",female,47.0,1,0,363272,7.0000,NaN,S
2,894,2,"Myles, Mr. Thomas Francis",male,62.0,0,0,240276,9.6875,NaN,Q
3,895,3,"Wirz, Mr. Albert",male,27.0,0,0,315154,8.6625,NaN,S
4,896,3,"Hirvonen, Mrs. Alexander (Helga E Lindqvist)",female,22.0,1,1,3101298,12.2875,NaN,S


## feature and target

In [6]:
target = 'Survived'
del_columns = ['Survived', 'Name']

features = list(set(train_df.columns) - set(del_columns))
features

['Cabin',
 'Ticket',
 'Age',
 'Embarked',
 'Sex',
 'PassengerId',
 'Fare',
 'SibSp',
 'Parch',
 'Pclass']

## train lgbm

### metrix

In [7]:
def evaluation(true, pred):
    return roc_auc_score(true, pred)

### encording

In [8]:
# 数値でないカラムをLabelEncording
categoricals = ['Sex', 'Ticket', 'Cabin','Embarked']
features = list(set(features) | set(categoricals))
for c in categoricals:
    le = LabelEncoder()
    le.fit(train_df.append(test_df)[c].astype(str))
    train_df[c] = le.transform(train_df[c].astype(str))
    test_df[c] = le.transform(test_df[c].astype(str))

# Count encoding
count_enc = ['Sex', 'Ticket', 'Cabin', 'Embarked']
count_features = []
for c in count_enc:
    _count = train_df.append(test_df)[c].astype(str).value_counts().to_dict()
    train_df[f'{c}_count'] = train_df[c].astype(str).map(_count)
    test_df[f'{c}_count'] = test_df[c].astype(str).map(_count)
    count_features.append(f'{c}_count')
features = list(set(features) | set(count_features))

### train

In [10]:
params = {
    'learning_rate': 0.01, 
    'loss_function': 'RMSE',
    'verbose':100,
    'iterations':5000,
    'early_stopping_rounds':100
}

# TargetEncording Config
target_enc = ['Sex', 'Ticket', 'Cabin', 'Embarked'] # 集計対象カラム
target_enc_key = ['Survived'] # 集計するキー

oof_preds = np.zeros(len(train_df))
y_pred = np.zeros(len(test_df))
models = []
cv_scores = {}
skf = StratifiedKFold(n_splits=Config.N_FOLD, random_state=Config.RANDOM_SATE, shuffle=True)
for fold, (train_index, test_index) in enumerate(skf.split(train_df[features], train_df[target])):

    print(f'====== fold {fold} ======')

    # TrainとTestに分割
    x_train, x_val = train_df.copy().iloc[train_index][features], train_df.copy().iloc[test_index][features]
    y_train, y_val =  train_df.iloc[train_index][target], train_df.iloc[test_index][target]

    test = test_df[features]

    # Target Encoding
    if len(target_enc) > 0:
        for t in target_enc_key:
            for c in target_enc:
                x_train[f'{c}_target_enc_by_{t}'] = train_df.iloc[train_index][c].map(train_df.iloc[test_index].groupby(c)[t].mean().to_dict())
                x_val[f'{c}_target_enc_by_{t}'] = train_df.iloc[test_index][c].map(train_df.groupby(c)[t].mean().to_dict())
                test[f'{c}_target_enc_by_{t}'] = test_df[c].map(train_df.groupby(c)[t].mean().to_dict())
            
    train_features = x_train.columns.to_list()
    
    # create Dataset
    train_pool = Pool(x_train, y_train, cat_features=categoricals)
    valid_pool = Pool(x_val, y_val, cat_features=categoricals)
    test_pool = Pool(test, cat_features=categoricals) 
    
    # train
    model = CatBoostRegressor(**params)
    model.fit(train_pool, eval_set=valid_pool)
    models.append(model)

    fold_pred = model.predict(valid_pool)
    
    score = evaluation(y_val, fold_pred)
    cv_scores[f'cv{fold}'] = score

    oof_preds[test_index] = fold_pred

    y_pred += model.predict(test_pool) / Config.N_FOLD

    print(f'cv score is {score}')

oof_score = evaluation(train_df[target], oof_preds)
print(f'OOF score is {oof_score}')


====== fold 0 ======
0:	learn: 0.4843345	test: 0.4848447	best: 0.4848447 (0)	total: 23.3ms	remaining: 23.2s
100:	learn: 0.3851203	test: 0.3922210	best: 0.3922210 (100)	total: 5.07s	remaining: 45.1s
200:	learn: 0.3559304	test: 0.3580515	best: 0.3580515 (200)	total: 9.51s	remaining: 37.8s
300:	learn: 0.3446871	test: 0.3443091	best: 0.3443091 (300)	total: 13.9s	remaining: 32.2s
400:	learn: 0.3374827	test: 0.3362220	best: 0.3362220 (400)	total: 17.6s	remaining: 26.4s
500:	learn: 0.3320833	test: 0.3309617	best: 0.3309617 (500)	total: 21.4s	remaining: 21.3s
600:	learn: 0.3276252	test: 0.3284914	best: 0.3284914 (600)	total: 25.2s	remaining: 16.8s
700:	learn: 0.3237087	test: 0.3267380	best: 0.3267019 (699)	total: 29.2s	remaining: 12.4s
800:	learn: 0.3202638	test: 0.3257331	best: 0.3256564 (796)	total: 33.2s	remaining: 8.25s
900:	learn: 0.3166407	test: 0.3254391	best: 0.3253654 (842)	total: 37.6s	remaining: 4.14s
999:	learn: 0.3135674	test: 0.3250076	best: 0.3249701 (991)	total: 42.6s	remaining